Easy
Write a function that takes a credit card number and only displays the last four characters. The rest of the card number must be replaced by ************.
Examples:

card_hide("1234123456785678") ➞ "************5678"

card_hide("8754456321113213") ➞ "************3213"

card_hide("35123413355523") ➞ "**********5523"

Notes:
Ensure you return a string.
The length of the string must remain the same as the input.

In [1]:
def card_hide(card):
	return '*'*len(card[:-4])+card[-4:]

print(card_hide("1234123456785678"))
print(card_hide("8754456321113213"))
print(card_hide("35123413355523"))

************5678
************3213
**********5523


Medium
Create a function that flips a horizontal list into a vertical list, and a vertical list into a horizontal list.
In other words, take an 1 x n list (1 row + n columns) and flip it into a n x 1 list (n rows and 1 column), and vice versa.

Examples:

`flip_list([1, 2, 3, 4]) ➞ [[1], [2], [3], [4]]`

Take a horizontal list and flip it vertical.

`flip_list([[5], [6], [9]]) ➞ [5, 6, 9]`

Take a vertical list and flip it horizontal.

`flip_list([]) ➞ []`

Notes:

If given an empty list [], return an empty list [].

In [2]:
def flip_list(lst):
	return [x[0] if isinstance(x, list) else [x] for x in lst]

print(flip_list([1, 2, 3, 4]))
print(flip_list([[5], [6], [9]]))
print(flip_list([]))

[[1], [2], [3], [4]]
[5, 6, 9]
[]


Hard
You will be given a polynomial expression in string form. The expression will contain any of the following operations, written using standard mathematical notation for a single variable, "x", as illustrated in the examples below:
addition: x + 1
subtraction: x – 2
multiplication: 3x
division: x / 4
exponentation: x^5
brackets: x(x + 1)
Your task is to write a function that can evaluate such a polynomial for a given value of x. You will receive two arguments: the polynomial string and the input number.
If the mathematical expression contains an error, you should return -1.
Examples:
```python
evalPolynomial("x+1", 5) ➞ 6
evalPolynomial("5x^2", 3) ➞ 45
evalPolynomial("(x(x+1))/2", 4) ➞ 10
evalPolynomial("4(x + 3))/2", 5) ➞ -1
```
// Invalid because parentheses not balanced.
Notes:
The string will not contain spaces.

In [29]:
import math
import re

class Node(object):
    def getResult(self):
        return 0

class ValNode(Node):
    def getResult(self):
        return int(self.val)

class OpNode(Node):

    val1: ValNode = None
    val2: ValNode = None
    opChar: str = None

    def __init__(self) -> None:
        super().__init__()

    def getResult(self):
        if self.opChar == "+":
            return int(self.val1.getResult() + self.val2.getResult())
        elif self.opChar == "-":
            return int(self.val1.getResult() - self.val2.getResult())
        elif self.opChar == "*":
            return int(self.val1.getResult() * self.val2.getResult())
        elif self.opChar == "/":
            return int(self.val1.getResult() / self.val2.getResult())
        elif self.opChar == "^":
            return int(math.pow(self.val1.getResult(), self.val2.getResult()))
        return 0

def isDigit(s: str) -> bool:
    return bool(re.match("[0-9]", s))

def containsChar(l: list[str], s: str) -> bool:
    return s in l

def buildNode(polyNode: str) -> OpNode:
    theNode: Node = None
    # addition and subtraction
    addSubSigns: list[str] = ["+", "-"]
    theNode = buildOpNode(polyNode, addSubSigns)
    if (theNode is not None):
        return theNode
    # multiplication and division
    mulDivSigns: list[str] = ["*", "/"]
    theNode = buildOpNode(polyNode, mulDivSigns)
    if (theNode is not None):
        return theNode
    # powers
    powSign: list[str] = ["^"]
    theNode = buildOpNode(polyNode, powSign)
    if (theNode is not None):
        return theNode
    # 
    if ((polyNode[0] == "(") and (polyNode[len(polyNode) - 1] == ")")):
        theNode = buildNode(polyNode[1:len(polyNode) - 1])
        return theNode
    if ((polyNode == "")):
        polyNode = "0"
    for s in polyNode:
        if not isDigit(s):
            return None
    theNode = ValNode()
    theNode.val = int(polyNode)
    return theNode

def buildOpNode(polyNode: str, ops: list[str]) -> OpNode:
    opNode: OpNode = None
    lastNode: OpNode = None
    brack: int = 0
    term: str = ""
    # iterate through the string of polynomials
    for s in polyNode:
        # if character is not an operational or in bracket
        if (not containsChar(ops, s)) or (brack != 0):
            term = term + s
        # iterate or deiterate depending if openning or closing bracket
        if s == "(":
            brack+=1
        if s == ")":
            brack-=1
        if brack != 0:
            continue
        if (containsChar(ops, s)):
            newNode: OpNode = OpNode()
            newNode.opChar = s
            newNode.val1 = buildNode(term)
            if lastNode is not None:
                lastNode.val2 = newNode
            lastNode = newNode
            if opNode is None:
                opNode = lastNode
            term = ""
    if lastNode is not None:
        lastNode.val2 = buildNode(term)
    return opNode

def evalPolynomial(poly: str, num: int) -> int:
    if poly == "":
        return -1
    correctedPoly: str = ""
    brack: int = 0
    for i in range(0, len(poly)):
        s: str = poly[i]
        # multiplication
        if (s == "x"):
            if i != 0:
                s2 = poly[i - 1]
                if isDigit(s2):
                    correctedPoly += "*"
            correctedPoly += str(num)
            continue
        # opening brackets 
        if s == "(":
            brack+=1
            if i != 0:
                s2: str = poly[i - 1]
                if (isDigit(s2) or (s2 == "x")):
                    correctedPoly += "*"
            correctedPoly += s
            continue
        # closing brackets
        if s == ")":
            brack-=1
            if i != (len(poly) - 1):
                s2: str = poly[i + 1]
                if isDigit(s2):
                    correctedPoly += s2 + "*"
                    continue
            correctedPoly += s
            continue
        correctedPoly += s

    if (brack != 0):
        return -1
    node: Node = buildNode(correctedPoly)
    if node is None:
        return -1
    res: int = node.getResult()
    return res


In [30]:
print(evalPolynomial("x+1", 5))
print(evalPolynomial("5x^2", 3))
print(evalPolynomial("(x(x+1))/2", 4))
print(evalPolynomial("4(x + 3))/2", 5))

6
45
10
-1
